# Inverse Kinematic example

Simple example for the inverse kinematics function of pybullet for following a trajectory.

In [1]:
# Import the needed packages
import numpy as np
import pybullet as pb
import pybullet_data as pbd
from datetime import datetime
from time import sleep

In [2]:
# Start the physic engine
client = pb.connect(pb.GUI)

# Add data path
pb.setAdditionalSearchPath(pbd.getDataPath())

In [3]:
# Reset the simulation to avoid multiple planes or kukas
pb.resetSimulation()

# Import a plane
plane = pb.loadURDF('plane.urdf')

# Import a kuka with a fixed base
kuka = pb.loadURDF('kuka_iiwa/model.urdf', useFixedBase = 1)
# Get the number of joint of the kuka
num_joints = pb.getNumJoints(kuka)
# Make an empty dictionary
joint_dict = {}
# Define a list of upper and lower limits
up_limits = []
low_limits = []
# Fill the dict
for i in range(num_joints):
    # Get a list of joint information
    joint_info = pb.getJointInfo(kuka, i)
    joint_dict.update({str(joint_info[1], 'utf-8') : joint_info[0]})
    # Get the limits
    up_limits.append(joint_info[9]), low_limits.append(joint_info[8])

# Print because safety first!
joint_dict, up_limits, low_limits

({'lbr_iiwa_joint_1': 0,
  'lbr_iiwa_joint_2': 1,
  'lbr_iiwa_joint_3': 2,
  'lbr_iiwa_joint_4': 3,
  'lbr_iiwa_joint_5': 4,
  'lbr_iiwa_joint_6': 5,
  'lbr_iiwa_joint_7': 6},
 [2.96705972839,
  2.09439510239,
  2.96705972839,
  2.09439510239,
  2.96705972839,
  2.09439510239,
  3.05432619099],
 [-2.96705972839,
  -2.09439510239,
  -2.96705972839,
  -2.09439510239,
  -2.96705972839,
  -2.09439510239,
  -3.05432619099])

In [6]:
# Define the start and stop time
start_time = 0
stop_time = 100
# Time step
dt = 0.0005

# Create a trajectory
# Array for time values
time = np.arange(start_time, stop_time, dt)
# Array of x,y,z, time values
x = np.zeros((3,len(time)))
for t in range(len(time)):
    x[:,t] = [0.3*np.sin(time[t]), 0.6*np.cos(time[t]), 0.5 + 0.25 *np.sin(10*time[t])]


# Set the simulation to step wise
pb.setRealTimeSimulation(0)
# Set the right timestep
pb.setTimeStep(dt)

for i in range(len(time)-1):
    # Get the current trajectory value
    current_x = x[:,i]
    # Get the next trajectory value
    next_x = x[:,i+1]
    # Add a line between the current value and the next value which lasts for some time
    pb.addUserDebugLine(current_x, next_x,lineColorRGB = [0, 1, 0],lifeTime = 20)
    # Calculate the inverse kinematics of the kuka robot for the last joint, the target position, with the limtis
    joint_pos = pb.calculateInverseKinematics(kuka, num_joints-1, next_x, lowerLimits = low_limits, upperLimits = up_limits)
    # Set the Kuka to Position control - maybe we can just directly set the joint states?
    pb.setJointMotorControlArray(kuka, np.arange(num_joints), controlMode = pb.POSITION_CONTROL, targetPositions = joint_pos, )
    # Calculate the position error because we can :)
    link_state = pb.getLinkState(kuka, num_joints-1)
    error = np.linalg.norm(current_x - np.array(link_state[0]))
    # add another line for the positional error
    pb.addUserDebugLine(current_x, next_x + np.array([0,0,error]), lineColorRGB = [1, 0, 0], lifeTime = 20)
    # Sleep to see something
    sleep(1e-3)
    # Step the simulation
    pb.stepSimulation()

KeyboardInterrupt: 